In [ ]:
!pip install tensorflow-datasets

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load COCO 2017 dataset with annotations
dataset, info = tfds.load('coco/2017', with_info=True, data_dir='/notebooks/tensorflow_datasets')
num_classes = 80 # number of classes in COCO dataset

In [ ]:
def create_datasets_from_coco(dataset, transform_image, num_classes):
    def encode_coco_categories(coco_categories, num_classes):
        return tf.reduce_max(tf.one_hot(coco_categories, num_classes), axis=0)

    def set_labels_as_zeros():
        return tf.zeros(num_classes, tf.float32)

    def preprocess(example, transform_image):
        image = example['image']
        image = transform_image(image)

        coco_categories = example['objects']['label']
        is_empty = tf.equal(tf.size(coco_categories), 0)
        labels = tf.cond(is_empty, set_labels_as_zeros, lambda: encode_coco_categories(coco_categories, num_classes))
        return image, labels

    batch_size = 32

    train_ds = dataset['train'].map(lambda x: preprocess(x, transform_image)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    val_ds = dataset['validation'].map(lambda x: preprocess(x, transform_image)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    # see a picture and its label
    # for image, label in train_ds.take(1):
    #     image_number = 3
    #     plt.imshow(image[image_number])
    #     # print(image[image_number][0])
    #     plt.show()
    #     print(label[image_number])
    return train_ds, val_ds

## Create your own model

In [ ]:
def transform_image(image):
    image = tf.image.resize(image, (227, 227))
    image = tf.cast(image, tf.float32) / 127.5 - 1.0
    mean = [0.485, 0.456, 0.406]
    variance = [0.229, 0.224, 0.225]
    normalization_layer = Normalization(mean=mean, variance=variance)
    image = normalization_layer(image)
    return image

train_ds, val_ds = create_datasets_from_coco(dataset, transform_image, num_classes)

input_shape = (227, 227, 3)

model = Sequential()
# Convolutional layers
model.add(Conv2D(96, (11, 11), strides=(4, 4), activation="relu", input_shape=input_shape, padding="same", kernel_initializer="he_normal"))
model.add(BatchNormalization())
model.add(MaxPooling2D((3, 3), strides=(2, 2)))
model.add(Conv2D(256, (5, 5), activation="relu", padding="same", kernel_initializer="he_normal"))
model.add(BatchNormalization())
model.add(MaxPooling2D((3, 3), strides=(2, 2)))
model.add(Conv2D(384, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal"))
model.add(BatchNormalization())
model.add(Conv2D(384, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal"))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal"))
model.add(BatchNormalization())
model.add(MaxPooling2D((3, 3), strides=(2, 2)))

# Fully connected layers
model.add(Flatten())
model.add(Dense(4096, activation="relu", kernel_initializer="glorot_normal"))
model.add(Dropout(0.5))
model.add(Dense(4096, activation="relu", kernel_initializer="glorot_normal"))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation="sigmoid"))

# Compile the model
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics='accuracy')

## Or load pretrained archtecture and change the last layer

In [ ]:
def transform_image(image):
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.vgg16.preprocess_input(image)
    return image
    
train_ds, val_ds = create_datasets_from_coco(dataset, transform_image, num_classes)

# Load the VGG16 model without the top layers
base_model = tf.keras.applications.VGG16(weights=None, include_top=False, input_shape=(224, 224, 3))

# Add top layers for multi-label classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='sigmoid')(x)

# Create the final model
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# Print the layer names
print(*(layer.name for layer in model.layers), sep=' -> ')

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Train and save

In [ ]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

model.save('model.h5')